[toc]

# Tensorflow tf.Variable 和 tf.get_variable

Tensorflow 创建变量主要与三个函数 `tf.Variable`，`tf.get_variable`，`tf.variable_scope`有关

## tf.variable_scope

`tf.variable_scope(name, reuse)` 

目的是创建一个变量空间，可以将之视为一个文件夹，里面储存相应在这个空间中创建的变量。

## tf.Variable

`tf.Variable(name, shape)` 创建一个新的变量，需要赋予初始值，可以不给名字，**如果当前命名空间中存在相同变量，会添加后缀 `_1`,`_2` 等来区分并再创建一个**。

In [28]:
import tensorflow as tf
tf.reset_default_graph()

a = tf.Variable(tf.ones([2,2]),name='c') # c:0
b = tf.Variable(tf.ones([2,2]),name='c') # c_1:0
    
with tf.variable_scope("foo"):
    a = tf.Variable(tf.ones([2,2]), name='c') # foo/c:0
    b = tf.Variable(tf.ones([2,2]), name='c') # foo/c_1:0
    
with tf.variable_scope("foo"):
    a = tf.Variable(tf.ones([2,2]),name='c') # foo_1/c:0

for var in tf.trainable_variables():
    print(var)

<tf.Variable 'c:0' shape=(2, 2) dtype=float32_ref>
<tf.Variable 'c_1:0' shape=(2, 2) dtype=float32_ref>
<tf.Variable 'foo/c:0' shape=(2, 2) dtype=float32_ref>
<tf.Variable 'foo/c_1:0' shape=(2, 2) dtype=float32_ref>
<tf.Variable 'foo_1/c:0' shape=(2, 2) dtype=float32_ref>


## tf.get_variable
    
`tf.get_variable(name, shape)` 必须要给一个名字，通常和 `tf.variable_scope` 一起使用。

1. 当 tf.variable_scope 的 reuse=False 时
    1. 如果 name 不存在于当前命名空间，tf.get_variable 会创建变量
    2. 如果 name 存在于当前命名空间，tf.get_variable 会报错 
    
    ```
    Variable xxx already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?
    ```
2. 当 tf.variable_scope 的 reuse=True 时
    1. 如果 name 不存在于当前命名空间，tf.get_variable 会报错
    ```
    Variable xxx does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?
    ```
    2.  如果 name 存在于当前命名空间，tf.get_variable 会获取这个变量的值
    
注意：这里同名的变量要求是之前用 `get_variable` 创建的，即无法获取用 `tf.Variable` 创建的同名向量。

In [29]:
import tensorflow as tf
tf.reset_default_graph()

a = tf.get_variable('c', [2,2]) # 'c:0'

with tf.variable_scope("foo"):
    b = tf.get_variable('c',[2,2]) # foo/c:0
    
with tf.variable_scope("foo", reuse=True):
    b = tf.get_variable('c',[2,2]) # 这里重用了 `foo/c:0`

for var in tf.trainable_variables():
    print(var)

<tf.Variable 'c:0' shape=(2, 2) dtype=float32_ref>
<tf.Variable 'foo/c:0' shape=(2, 2) dtype=float32_ref>


如果将上面的 `reuse=True` 删除，会报错：

```
Variable foo/c already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?
```

# References
1. [tensorflow笔记 变量_thormas1996的博客-CSDN博客](https://blog.csdn.net/thormas1996/article/details/81129379)